# Análisis Exploratorio de Datos - Clientes Bancarios
## Este notebook realiza el análisis exploratorio de datos de un dataset de clientes bancarios con Python

## 1. Importación de librerías y carga de datos

In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización
plt.style.use('default')
sns.set_theme()
%matplotlib inline

# Importar funciones de carga de datos
import sys
sys.path.append('../src')
from data.data_loader import load_customer_data, load_bank_data, get_basic_info

In [2]:
# Cargar datos de clientes
df_customers = load_customer_data()

# Cargar datos bancarios
df_bank = load_bank_data()

In [3]:
# Información básica de los datos de clientes
print("=== Información de Datos de Clientes ===")
get_basic_info(df_customers)

# Información básica de los datos bancarios
print("\n=== Información de Datos Bancarios ===")
get_basic_info(df_bank)

=== Información de Datos de Clientes ===

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 20115 entries, 0 to 20114
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Income             20115 non-null  int64         
 1   Kidhome            20115 non-null  int64         
 2   Teenhome           20115 non-null  int64         
 3   Dt_Customer        20115 non-null  datetime64[ns]
 4   NumWebVisitsMonth  20115 non-null  int64         
 5   ID                 20115 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 1.1+ MB
None

Primeras 5 filas:
   Income  Kidhome  Teenhome Dt_Customer  NumWebVisitsMonth  \
0  161770        1         0  2012-04-04                 29   
1   85477        1         1  2012-12-30                  7   
2  147233        1         1  2012-02-02                  5   
3  121393        1         2  2012-12-21   

## 2. Transformación y limpieza de los datos

### 2.1 Estandarización de nombres de columnas

In [4]:
# Mostrar columnas de cada dataset
print("Columnas en datos de clientes:")
print(df_customers.columns.tolist())

print("\nColumnas en datos bancarios:")
print(df_bank.columns.tolist())

Columnas en datos de clientes:
['Income', 'Kidhome', 'Teenhome', 'Dt_Customer', 'NumWebVisitsMonth', 'ID']

Columnas en datos bancarios:
['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y', 'date', 'latitude', 'longitude', 'id_']


In [5]:
# Rename de las columnas para una mejor comprensión de su significado, ya que algunas columnas presentan un exceso de abreviación
# Renombrar columnas del dataset de clientes
customer_columns_rename = {
    'Income': 'Income',
    'Kidhome': 'Number_of_Children',
    'Teenhome': 'Number_of_Teenagers',
    'Dt_Customer': 'Registration_Date',
    'NumWebVisitsMonth': 'Monthly_Web_Visits',
    'ID': 'Customer_ID'
}

# Renombrar columnas del dataset bancario
bank_columns_rename = {
    'age': 'Age',
    'job': 'Job',
    'marital': 'Marital_Status',
    'education': 'Education_Level',
    'default': 'Credit_Default',
    'housing': 'Mortgage_Loan',
    'loan': 'Personal_Loan',
    'contact': 'Contact_Type',
    'duration': 'Call_Duration',
    'campaign': 'Campaign_Contacts',
    'pdays': 'Days_Since_Last_Contact',
    'previous': 'Previous_Contacts',
    'poutcome': 'Previous_Campaign_Outcome',
    'emp.var.rate': 'Employment_Variation_Rate',
    'cons.price.idx': 'Consumer_Price_Index',
    'cons.conf.idx': 'Consumer_Confidence_Index',
    'euribor3m': 'Euribor_3M_Rate',
    'nr.employed': 'Number_of_Employees',
    'y': 'Subscribed_to_Service',
    'date': 'Date',
    'latitude': 'Latitude',
    'longitude': 'Longitude',
    'id_': 'Customer_ID'
}

# Aplicar los cambios
df_customers = df_customers.rename(columns=customer_columns_rename)
df_bank = df_bank.rename(columns=bank_columns_rename)

# Verificar los cambios
print("Columnas en datos de clientes después del rename:")
print(df_customers.columns.tolist())
print("\nColumnas en datos bancarios después del rename:")
print(df_bank.columns.tolist())

Columnas en datos de clientes después del rename:
['Income', 'Number_of_Children', 'Number_of_Teenagers', 'Registration_Date', 'Monthly_Web_Visits', 'Customer_ID']

Columnas en datos bancarios después del rename:
['Age', 'Job', 'Marital_Status', 'Education_Level', 'Credit_Default', 'Mortgage_Loan', 'Personal_Loan', 'Contact_Type', 'Call_Duration', 'Campaign_Contacts', 'Days_Since_Last_Contact', 'Previous_Contacts', 'Previous_Campaign_Outcome', 'Employment_Variation_Rate', 'Consumer_Price_Index', 'Consumer_Confidence_Index', 'Euribor_3M_Rate', 'Number_of_Employees', 'Subscribed_to_Service', 'Date', 'Latitude', 'Longitude', 'Customer_ID']


### 2.2 Corrección de formato y transformación de columnas

In [ ]:
# rhs

### 2.3 Análisis de valores nulos

In [6]:
# Análisis de valores nulos en datos de clientes
print("=== Valores Nulos en Datos de Clientes ===")
null_customers = df_customers.isnull().sum()
print(null_customers[null_customers > 0])

# Análisis de valores nulos en datos bancarios
print("\n=== Valores Nulos en Datos Bancarios ===")
null_bank = df_bank.isnull().sum()
print(null_bank[null_bank > 0])

=== Valores Nulos en Datos de Clientes ===
Series([], dtype: int64)

=== Valores Nulos en Datos Bancarios ===
Age                     5120
Job                      345
Marital_Status            85
Education_Level         1807
Credit_Default          8981
Mortgage_Loan           1026
Personal_Loan           1026
Consumer_Price_Index     471
Euribor_3M_Rate         9256
Date                     248
dtype: int64


### 2.4 Tratado de los nulos

In [7]:
# Detectar tipos de columnas
categorical_cols = ['Job', 'Marital_Status', 'Education_Level', 'Credit_Default', 'Mortgage_Loan', 'Personal_Loan']
numeric_cols = ['Age', 'Consumer_Price_Index', 'Euribor_3M_Rate']
date_cols = ['Date']

# === Reemplazo de nulos en columnas categóricas con "Unknown"
df_bank[categorical_cols] = df_bank[categorical_cols].fillna("Unknown")

# === Reemplazo de nulos en columnas numéricas con la media de la columna
for col in numeric_cols:
    mean_value = df_bank[col].mean()
    df_bank[col] = df_bank[col].fillna(mean_value)

# === Reemplazo de nulos en columnas de fecha con la media de la fecha
# Asegurar que sea datetime, errors='coerce' convierte strings invalidos a NaT (NaN para fechas)
df_bank['Date'] = pd.to_datetime(df_bank['Date'], errors='coerce')

# Calcular la media temporal (convertir a timestamp -> sacar media -> volver a datetime)
mean_timestamp = df_bank['Date'].dropna().astype(np.int64).mean()
mean_date = pd.to_datetime(mean_timestamp)

# Reemplazar los nulos por la fecha promedio
df_bank['Date'] = df_bank['Date'].fillna(mean_date)


# Verifica que no queden nulos
print(df_bank.isnull().sum().sort_values(ascending=False))

TypeError: can only concatenate str (not "int") to str

In [ ]:
# Información básica de los datos de clientes
print("=== Información de Datos de Clientes ===")
get_basic_info(df_customers)

# Información básica de los datos bancarios
print("\n=== Información de Datos Bancarios ===")
get_basic_info(df_bank)